## Web Scraping

In this section:
1. Fetch the Wikipedia page containing the list of highest-grossing films.
2. Locate the table with the film data.
3. Extract the required fields (title, release year, director, box office, and country).

In [1]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import json
import re
import sqlite3


In [2]:
# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"

# Fetch the page content
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Locate the table containing the highest-grossing films
# The table is usually the first one on the page with class "wikitable"
table = soup.find('table', {'class': 'wikitable'})

# Initialize a list to store the extracted data
films = []

def fix_director_names(director):
    # Insert space between lowercase and uppercase letters
    director = re.sub(r'([a-z])([A-Z])', r'\1 \2', director)
    return director


def parse_box_office(box_office):
    """
    Converts box office strings like "$1.23 billion" to a formatted string with the currency symbol.
    Example: "$1.23 billion" -> "$1230000000"
    """
    if not box_office or box_office == "N/A":
        return "N/A"

    # Extract currency symbol (assuming it's the first character)
    currency = box_office[0] if box_office[0] in "$€£¥" else ""

    # Extract numeric value and multiplier (e.g., "billion", "million")
    match = re.search(r'([\d.]+)\s*(million|billion)?', box_office, re.IGNORECASE)
    if not match:
        return "N/A"

    value = float(match.group(1))
    multiplier = match.group(2)

    # Convert to numeric value
    if multiplier and "billion" in multiplier.lower():
        value *= 1_000_000_000
    elif multiplier and "million" in multiplier.lower():
        value *= 1_000_000

    # Format the value as a string with the currency symbol
    return f"{currency}{int(value)}"


# Iterate through the table rows (skip the header row)
for row in table.find_all('tr')[1:]:
    columns = row.find_all({'td','th'})

    # Extract the required fields

    title = columns[2].text.strip()
    release_year = columns[4].text.strip()
    # Initialize missing fields

    director = "N/A"
    box_office = "N/A"
    country = "N/A"

    # Extract the link to the film's individual Wikipedia page
    film_link = columns[2].find('a')['href'] if columns[2].find('a') else None

    if film_link:
        # Fetch the film's individual page
        film_url = f"https://en.wikipedia.org{film_link}"
        film_response = requests.get(film_url)
        film_soup = BeautifulSoup(film_response.text, 'html.parser')


        infobox = film_soup.find('table', {'class': 'infobox'})

        if infobox:
            for row in infobox.find_all('tr'):
                header = row.find('th')
                if header:
                    header_text = header.text.strip().lower()
                    if 'directed by' in header_text:
                        director = row.find('td').text.strip()
                    if 'countries' in header_text or 'country' in header_text:
                        country = row.find('td').text.strip()
                    if 'box office' in header_text:
                        box_office = row.find('td').text.strip()

    # Append the extracted data to the list
    films.append({
        'title': title,
        'release_year': release_year,
        'director': director,
        'box_office': box_office,
        'country': country
    })

    # Clean the data
    pattern = r'\[\d+\]|\\n|†|US'
    for film in films:
      for key, val in film.items():
          cleaned_val = re.sub(pattern, '', val)
          cleaned_val = ' '.join(cleaned_val.split()).strip()
          film[key]=cleaned_val
    for film in films:
      film['director'] = fix_director_names(film['director'])
      film['box_office'] = parse_box_office(film['box_office'])


# Save the extracted data to a JSON file
with open('highest_grossing_films.json', 'w', encoding='utf-8') as f:
    json.dump(films, f, ensure_ascii=False, indent=4)


## Data Storage

In this section I store the data in a SQLite database for further analysis.

In [3]:
# Connect to a SQLite database (or create it if it doesn't exist)
db_file = "highest_grossing_films.db"
conn = sqlite3.connect(db_file)
cursor = conn.cursor()


In [4]:
# Create a table to store the film data
cursor.execute("""
CREATE TABLE highest_grossing_films (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    release_year INTEGER,
    director TEXT,
    box_office TEXT,
    country TEXT
)
""")
conn.commit()  # Save the changes to the database


In [5]:
# Prepare the data for insertion into the database
films_to_insert = []
for film in films:
    films_to_insert.append((
        film['title'],
        film['release_year'],
        film['director'],
        film['box_office'],
        film['country']
    ))

# Insert the data into the database
cursor.executemany("INSERT INTO highest_grossing_films (title, release_year, director, box_office, country) VALUES (?, ?, ?, ?, ?)", films_to_insert)
conn.commit()
conn.close()

